# 安裝

In [ ]:
!pip install yfinance ta
!pip install gymnasium --upgrade
!pip install stable-baselines3 --upgrade
!pip install torch shimmy

# 載入

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import ta
import stable_baselines3
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv
import gymnasium as gym
from gym import Env,spaces

ModuleNotFoundError: No module named 'gymnasium.wrappers.monitoring'

# 資料取得

In [ ]:
# 股票代碼和期間
tickers = ['META', 'AAPL', 'AMZN', 'MSFT', 'GOOG']
start_date = '2010-01-01'
end_date = '2020-12-31'

# 下載股票數據
data = yf.download(tickers, start=start_date, end=end_date)

# 獲取 Adj Close、High、Low
close_data = data['Adj Close']
high_data = data['High']
low_data = data['Low']

# 計算每日收益率和對數收益率
daily_return = close_data.pct_change()
log_return = np.log(close_data / close_data.shift(1))

# 每日收益率和對數收益率添加到 DataFrame 中
close_data = pd.concat(
    [close_data, daily_return.add_suffix('_Daily Return'), log_return.add_suffix('_Log Return')],
    axis=1
)

[*********************100%***********************]  5 of 5 completed


# 設計變數


In [ ]:
# 每支股票計算技術指標
for ticker in tickers:
    # 移動平均線 (50天窗口)
    close_data[f'{ticker}_MA'] = close_data[ticker].rolling(window=50).mean()

    # 相對強弱指數 (RSI, 14天窗口)
    close_data[f'{ticker}_RSI'] = ta.momentum.RSIIndicator(close_data[ticker], window=14).rsi()

    # 順勢指數 (CCI, 20天窗口)
    close_data[f'{ticker}_CCI'] = ta.trend.CCIIndicator(high=high_data[ticker], low=low_data[ticker], close=close_data[ticker], window=20).cci()

    # 平均趨向指數 (ADX)
    close_data[f'{ticker}_ADX'] = ta.trend.ADXIndicator(high=high_data[ticker], low=low_data[ticker], close=close_data[ticker]).adx()

    # 布林通道 (20天窗口)
    bb = ta.volatility.BollingerBands(close_data[ticker], window=20)
    close_data[f'{ticker}_BB High'] = bb.bollinger_hband()
    close_data[f'{ticker}_BB Low'] = bb.bollinger_lband()

    # MACD (標準窗口)
    macd = ta.trend.MACD(close_data[ticker])
    close_data[f'{ticker}_MACD'] = macd.macd()
    close_data[f'{ticker}_MACD Signal'] = macd.macd_signal()

# 協方差矩陣 (使用百分比變化计算協方差)
cov_matrix = close_data[[ticker for ticker in tickers]].pct_change().cov()

# 删除缺失值
close_data = close_data.dropna()

# 切分資料集

In [ ]:
# 80% 訓練集、20% 測試集
#len(close_data) 總行數
#int(...) 轉換為整數
train_size = int(len(close_data) * 0.8)
train_data = close_data.iloc[:train_size]
test_data = close_data.iloc[train_size:]

# 定義投資組合環境
該環境模擬了一個簡單的投資組合管理過程，允許智能體在給定的市場數據下學習如何分配資產權重以最大化其投資reset回報step。計算獎勵。

In [ ]:
class PortfolioEnv(gym.Env):
    def __init__(self, data):#初始化
        super(PortfolioEnv, self).__init__()
        self.data = data
        self.n_assets = len(tickers)
        #規定安裝的操作空間，表示每個資產的投資比例，範圍從0到1
        self.action_space = spaces.Box(low=0.0, high=1.0, shape=(self.n_assets,), dtype=np.float32)
        #定義觀察空間，表示每個時間步驟的觀察內容，包括資產價格和投資組合權重
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets * 2,), dtype=np.float32)

        self.start_index = 0
        self.current_step = self.start_index
        self.initial_balance = 1000000 #初始的資金餘額
        self.balance = self.initial_balance
        self.portfolio_weights = np.array([1 / self.n_assets] * self.n_assets)

    def reset(self, seed=None, **kwargs):#重置環境(當前步數、餘額和投資組合權重)
        self.current_step = self.start_index
        self.balance = self.initial_balance
        self.portfolio_weights = np.array([1 / self.n_assets] * self.n_assets)
        return self.get_obs(), {}  #傳回觀察結果和一個空的資訊字典

    def step(self, action):#行進方式
        #標準化權重
        action = action / np.sum(action)
        self.portfolio_weights = action

        #執行到最後一個資料，回傳觀察值、獎勵和結束標誌
        if self.current_step >= len(self.data) - 1:
            done = True
            obs = self.get_obs()
            return obs, 0, done, {}
        #計算收益
        returns = self.data.iloc[self.current_step + 1][tickers].pct_change().values
        portfolio_return = np.dot(self.portfolio_weights, returns)
        #計算獎勵並更新餘額
        reward = self.balance * portfolio_return
        self.balance += reward
        #更新時間步數並判斷是否結束
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        #回傳觀察值、獎勵和結束標誌
        obs = self.get_obs()
        return obs, reward, done, {}

    def get_obs(self):#獲取觀察值方法(收集當前時間步每個資產的價格、將目前的投資組合權重加到觀察值中)
        obs = []
        for ticker in tickers:
            obs.append(self.data[ticker].iloc[self.current_step] if self.current_step < len(self.data) else 0)
        obs.extend(self.portfolio_weights)
        return np.array(obs)

# 創建訓練環境

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor

# 創建訓練環境
train_env = PortfolioEnv(train_data)
# 監控環境(記錄智能體在每次（訓練輪次）中 reward、steps)
train_env = Monitor(train_env)
# 支持化環境
# lambda: train_env每個函數傳回一個新的環境實例
train_env = DummyVecEnv([lambda: train_env])

# 創建SAC模型、訓練模型

In [ ]:
#'MlpPolicy'是指使用一個梯度增長器（MLP）來選擇智能體在環境中每一步應該採取的行動
model = SAC('MlpPolicy', env, verbose=1)
# 訓練過程會進行 10,000 個時間
model.learn(total_timesteps=10000)

Using cpu device
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 42       |
|    time_elapsed    | 18       |
|    total_timesteps | 800      |
| train/             |          |
|    actor_loss      | 25.3     |
|    critic_loss     | 0.226    |
|    ent_coef        | 0.813    |
|    ent_coef_loss   | -0.325   |
|    learning_rate   | 0.0003   |
|    n_updates       | 699      |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 38       |
|    time_elapsed    | 41       |
|    total_timesteps | 1600     |
| train/             |          |
|    actor_loss      | 51.6     |
|    critic_loss     | 0.133    |
|    ent_coef        | 0.647    |
|    ent_coef_loss   | -0.61    |
|    learning_rate   | 0.0003   |
|    n_updates       | 1499     |
---------------------------------
-------------------------------

# 建立測試環境

In [ ]:
test_env = Monitor(PortfolioEnv(test_data))
test_env = DummyVecEnv([lambda: test_env])

# 評估模型

In [ ]:
total_rewards = 0

#循環進行預測與環境步進
for _ in range(len(test_data) - 1):
    action, _ = model.predict(obs)
    obs, reward, done, _ = test_env.step(action)
    total_rewards += reward
    if done:
        break

print(f'Total Rewards: {total_rewards}')
print(f'Final Balance: {underlying_env.balance}')

Total Rewards: 0
Final Balance: 1000000


#結果探討
**超參數調整**
軟性演員評論家演算法 (SAC)
Reward Scale 是 SAC 特有的超參數,該參數意涵是指直接讓獎勵(Reward) 乘以一個常數(Reward Scale,k),在不破壞獎勵函數的前提下調整獎勵值,從而間接調整 Q 值到合適的水準,∑kri = k∑ri =kQt,
其中 Q 為累積收益,本文使用 k 為 1000,該數字是依經驗進行調整的適當水準,原論文已有論述僅需讓累計收益的範圍落在正負 1000以內即可,不需要精細調整

儘管如此它仍為本模型最關鍵的超參數,該參數越大隱含更少的 Entropy,將使得 SAC 模型漸近於 DDPG,該參數越小,將使的策略分布趨近於 Uniform,代表此將不利於探索。

**變數調整**
本身該資料集報酬為負，導致無法決策且有效學習。



1.模型未訓練充分：
訓練次數（total_timesteps）可能使模型學習達到有效的策略。

試著增加total_timesteps數值，讓模型有更多的時間學習有效的策略。

2.模型複雜度：
MLP（多層採集器）的結構可能不夠複雜，無法捕捉資料中的潛在模式。

考慮增加MLP的層數或每層的神經元數量，以提高模型的表達能力。

3.測試資料特性：
資料可能與訓練資料差異增大，導致模型在未見過的測試情況下表現不佳。

在測試模型之前，先在訓練資料上進行調試，確保模型能夠在訓練資料中獲得合理的獎勵。

改進方法
增加訓練時間：

試著增加total_timesteps數值，讓模型有更多的時間學習有效的策略。
調整獎勵機制：

重新設計獎勵結構，考慮引入更多的獎勵訊號，例如交易的成功率、波動率等，以便模型能學會平衡風險與效益。
查詢資料品質：

輸入的資料沒有缺失值或異常值，並且具有足夠的時間跨度和多樣性，確保模型能夠學習。
優化模型結構：

考慮增加MLP的層數或每層的神經元數量，以提高模型的表達能力。
調整動作空間：

重新安排行動歸一化的流程，確保模型能夠探索有效的投資組合。
使用訓練資料進行調試：

在測試模型之前，先在訓練資料上進行調試，確保模型能夠在訓練資料中獲得合理的獎勵。
增加探索性：

調整演算法的超參數，例如增加探索率，以便模型能夠嘗試不同的投資策略。